### NAVER

In [3]:
import requests
import sqlite3

In [4]:
from bs4 import BeautifulSoup

In [5]:
from selenium import webdriver

In [6]:
import re

#### 링크 모으기

In [7]:
driver = webdriver.Chrome()

In [12]:
def collectLinks(url):
    driver.get(url)
    curPage = 0
    links = []
    pages = driver.find_elements_by_css_selector('.paging a')
    pages = [page.get_attribute('href') for page in pages] #2부터
    while curPage < len(pages):
        obj = driver.find_elements_by_css_selector(".type06_headline dt a")        
        tempLinks = [link.get_attribute('href') for link in obj]
        links.extend(tempLinks)
        driver.get(pages[curPage])
        curPage += 1
        if curPage==10:
            curPage = 0
            pages = driver.find_elements_by_css_selector('.paging a')
            pages = [page.get_attribute('href') for page in pages]
            pages = pages[1:]
    return links

#### 해당 링크에서 뷰티풀솝으로 파싱

In [40]:
def parsingArticleReq(link, headers, sect):
    try:
        article = requests.get(link, headers=headers)
    except:
        article = requests.get(link, headers=headers, verify=False)
    article = BeautifulSoup(article.text, "lxml")
    url = link
    a_id = link.split('aid=')
    a_id = "na_"+a_id[1]
    try:
        articleTitle = article.select_one("#articleTitle")
    except:
        return
    else:
        articleTitle = articleTitle.text.strip()
        dateTime = article.select(".t11")
        dateTime = dateTime[0].text.split(" ")
        date = dateTime[0]
        time = dateTime[1]
        reg_ex = {'email':r'[a-zA-Z0-9.!#$%&\'*+/=?^_`{|}~-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+'}
        cont = article.select_one("#articleBodyContents").text
        # content : 기자 이메일 직전 개행문자부터 이하 내용 제거
        try:
            email_idx = re.search(reg_ex['email'], cont).start() #이메일 시작 idx
            content_temp = cont[:email_idx].strip()
            if (len(content_temp) / len(cont)) > 0.6:
                cont = content_temp
        except:
             pass
        cont = cont.replace("// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}","").strip()
        tempIdx = cont.index('▶')
        if tempIdx > 0:
            cont = cont[:tempIdx]
        res = re.search(r"\w+\s+기자",cont)
        if res != None:
            author = res.group()
            author = author.replace(' 기자','')
        else:
            author = str()
        press = article.select_one(".press_logo a img")
        press = press.get('title')
        section = sect
        data = (a_id, date, time, articleTitle, cont, press, author, section, url)
        try:
            cur.execute("INSERT INTO naver(a_ids, dates, times, titles, contents, press, authors, sections, urls) \
                        values(?,?,?,?,?,?,?,?,?)", data)
        except:
            data = ''
        else:
            conn.commit()

#### 다른포맷

In [41]:
def diffFormatCrawling(link, headers, sect):
    try:
        article = requests.get(link, headers=headers)
    except:
        article = requests.get(link, headers=headers, verify=False)
    article = BeautifulSoup(article.text, "lxml")
    url = link
    a_id = link.split('aid=')
    a_id = "na_"+a_id[1]
    articleTitle = article.select_one('title').text.strip()
    dateTime = article.select('.info span')[1].text
    dateTime = dateTime.split(" ")
    date = dateTime[1]
    time = dateTime[2]+" "+dateTime[3]
    reg_ex = {'email':r'[a-zA-Z0-9.!#$%&\'*+/=?^_`{|}~-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+'}
    cont = article.select_one("#newsEndContents").text
    # content : 기자 이메일 직전 개행문자부터 이하 내용 제거
    try:
        email_idx = re.search(reg_ex['email'], cont).start() #이메일 시작 idx
        content_temp = cont[:email_idx].strip()
        if (len(content_temp) / len(cont)) > 0.6:
            cont = content_temp
    except:
        pass
    cont = cont.replace("// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}","").strip()
    tempIdx = cont.index('▶')
    if tempIdx > 0:
        cont = cont[:tempIdx]
    res = re.search(r"\w+\s+기자",cont)
    if res != None:
        author = res.group()
        author = author.replace(' 기자','')
    else:
        author = str()
    press = article.select_one(".link img")
    press = press.get('alt')
    section = sect
    data = (a_id, date, time, articleTitle, cont, press, author, section, url)
    try:
        cur.execute("INSERT INTO naver(a_ids, dates, times, titles, contents, press, authors, sections, urls) \
                        values(?,?,?,?,?,?,?,?,?)", data)
    except:
        pass
    else:
        conn.commit()

#### 0811 부분 링크긁기

In [38]:
url = 'https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm'
sid = {'politics':'100','economic':'101','society':'102','culture':'103','globalList':'104','digital':'105'}

politics = ['264','265','266','267','268','269']
economic = ['259','258','261','771','260','262','310','263']
society = ['249','250','251','254','252','59b','255','256','276','257']
globalList = ['231','232','233','234','322']
culture = ['241','239','240','237','238','376','242','243','244','248','245']
digital = ['731','226','227','230','732','283','229','228']

politicsLink = []
economicLink = []
societyLink = []
globalLink = []
cultureLink = []
digitalLink = []

for sid_k,sid_v in sid.items():
    if sid_k == 'politics':
        for t in politics:
            temp = collectLinks(url+"&sid1="+sid_v+"&sid2="+t+"&date=20180813")
            politicsLink.extend(temp)
    elif sid_k == 'economic':
        for t in economic:
            temp = collectLinks(url+"&sid1="+sid_v+"&sid2="+t+"&date=20180813")
            economicLink.extend(temp)
    elif sid_k == 'culture':
        for t in culture:
            temp = collectLinks(url+"&sid1="+sid_v+"&sid2="+t+"&date=20180813")
            cultureLink.extend(temp)
    elif sid_k == 'society':
        for t in society:
            temp = collectLinks(url+"&sid1="+sid_v+"&sid2="+t+"&date=20180813")
            societyLink.extend(temp)
    elif sid_k == 'globalList':
        for t in globalList:
            temp = collectLinks(url+"&sid1="+sid_v+"&sid2="+t+"&date=20180813")
            globalLink.extend(temp)
    else:
        for t in digital:
            temp = collectLinks(url+"&sid1="+sid_v+"&sid2="+t+"&date=20180813")
            digitalLink.extend(temp)

#### DB만들기

In [8]:
conn = sqlite3.connect('naver.db')

cur = conn.cursor()

In [75]:
cur.execute("CREATE TABLE naver(a_ids TEXT primary key, \
            dates DATE, times TIME, titles TEXT, contents TEXT, press TEXT, authors TEXT, sections TEXT, urls TEXT)")

In [ ]:
#cur.execute("DELETE FROM naver")

In [30]:
import pickle

def savePickle(date):
    with open('C:/Users/hanki/Desktop/빅데이터/newStory_git/newstudy/db/urls/naver/naver_politics_' + date + '.pkl', 'wb') as f:
        pickle.dump(politicsLink,f)
    with open('C:/Users/hanki/Desktop/빅데이터/newStory_git/newstudy/db/urls/naver/naver_economic_' + date + '.pkl', 'wb') as f:
        pickle.dump(economicLink,f)
    with open('C:/Users/hanki/Desktop/빅데이터/newStory_git/newstudy/db/urls/naver/naver_society_'+ date + '.pkl', 'wb') as f:
        pickle.dump(societyLink,f)
    with open('C:/Users/hanki/Desktop/빅데이터/newStory_git/newstudy/db/urls/naver/naver_global_' + date + '.pkl', 'wb') as f:
        pickle.dump(globalLink,f)
    with open('C:/Users/hanki/Desktop/빅데이터/newStory_git/newstudy/db/urls/naver/naver_culture_'+ date + '.pkl', 'wb') as f:
        pickle.dump(cultureLink,f)
    with open('C:/Users/hanki/Desktop/빅데이터/newStory_git/newstudy/db/urls/naver/naver_digital_'+ date + '.pkl', 'wb') as f:
        pickle.dump(digitalLink,f)

In [39]:
savePickle('20180813')

In [ ]:
with open('linkPickle/' + 'pickle_naver0810.pkl', 'rb') as f:
    test = pickle.load(f)
    
test[:10]

#### 긁어온 링크에서 파싱

In [10]:
headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36",
}

In [ ]:
for p in politicsLink:
    try:
        parsingArticleReq(p, headers, '정치')
    except:
#        diffFormatCrawling(p, headers, '정치')
        try:
            diffFormatCrawling(p, headers, '정치')
        except:
            print(p)

In [ ]:
for e in economicLink:
    try:
        parsingArticleReq(e, headers, '경제')
    except:
        try:
            diffFormatCrawling(e, headers, '경제')
        except:
            print(e)

In [ ]:
for s in societyLink:
    try:
        parsingArticleReq(s, headers, '사회')
    except:
        try:
            diffFormatCrawling(s, headers, '사회')
        except:
            print(s)

In [ ]:
for c in cultureLink:
    try:
        parsingArticleReq(c, headers, '문화')
    except:
        try:
            diffFormatCrawling(c, headers, '사회')
        except:
            print(c)

In [ ]:
for d in digitalLink:
    try:
        parsingArticleReq(d, headers, 'IT')
    except:
        try:
            diffFormatCrawling(d, headers, 'IT')
        except:
            print(d)    

In [49]:
for g in globalLink:
    try:
        parsingArticleReq(g, headers, '세계')
    except:
        try:
            diffFormatCrawling(g, headers, '세계')
        except:
            print(g)

https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=104&sid2=232&oid=001&aid=0010281833
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=104&sid2=232&oid=001&aid=0010281833
